# Gibberish Detector
## "Gibberish" vs. "Substantive"
- **Gibberish:** Random strings (e.g., "asdfjkl"), typos ("jhkd"), or meaningless repetition ("aaa").
- **Substantive:** Coherent sentences with intent (e.g., "Great course, learned a lot" or "这是一个很好的课程").
- **Edge Cases:** Short but valid reviews (e.g., "Good"), multilingual mixes, or sarcasm.

In [1]:
!pip -q install transformers huggingface_hub langdetect pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.1 MB/s eta 0:00:00:00:0100:01


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import scipy.stats as stats
import re
import nltk
from nltk.corpus import words
from collections import Counter
import math
import unicodedata
import time
from tqdm import tqdm
import pickle
from langdetect import detect
import warnings
import pycountry
from scipy import stats
from collections import defaultdict

---------------------BEGIN DATA IMPORT------------

### Importing Data
- The original data used an older character encoding
- We'll save them as utf-8 so we can add the csv's to our project dataset

In [8]:
# Try different encodings
file_path = '/kaggle/input/gibberish-text-classification/Amazon.csv'  # Replace with your file path
try:
    df = pd.read_csv(file_path, encoding='utf-8')  # Default, might fail
except UnicodeDecodeError:
    print("UTF-8 failed, trying other encodings...")
    try:
        df = pd.read_csv(file_path, encoding='windows-1252')
        print("Loaded with Windows-1252")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='iso-8859-1')
            print("Loaded with ISO-8859-1")
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='utf-16')
                print("Loaded with UTF-16")
            except UnicodeDecodeError:
                print("All common encodings failed. Check file encoding or corruption.")

UTF-8 failed, trying other encodings...
Loaded with Windows-1252


In [11]:
# Load with Windows-1252, save as UTF-8 for adding to our custom kaggle dataset
def convert_encoding_type(file, out):
    # Convert data to more universal encoding type
    df = pd.read_csv(file, encoding='windows-1252')
    df.to_csv(out, encoding='utf-8', index=False)
    print(f"Converted and saved as {out}")
    return

convert_encoding_type('/kaggle/input/gibberish-text-classification/Amazon.csv', 'amazon_reviews.csv')
convert_encoding_type('/kaggle/input/gibberish-text-classification/Gibberish.csv', 'amazon_gibberish.csv')

Converted and saved as amazon_reviews.csv
Converted and saved as amazon_gibberish.csv


-----------------------------END DATA CREATION---------------------

## EDA and Feature Extraction

In [7]:
reviews_df = pd.read_csv('/kaggle/input/gibberish-text-classification/Amazon.csv', encoding='windows-1252', header=None, names=['rating', 'review'])
gibb_df = pd.read_csv('/kaggle/input/gibberish-text-classification/Gibberish.csv', encoding='windows-1252')

According to the original data set at https://www.kaggle.com/datasets/bittlingmayer/amazonreviews, __label__2 means 4 and 5 star product reviews and label 1 is 1 and 2 start reviews. Looks like there's roughly equal amounts of both (positive and negative sentiment)

In [8]:
reviews_df.rating.value_counts()

rating
__label__2    530161
__label__1    518415
Name: count, dtype: int64

In [9]:
reviews_df.head()

,rating,review
0,__label__2,Stuning even for the non-gamer: This sound tra...
1,__label__2,The best soundtrack ever to anything.: I'm rea...
2,__label__2,Amazing!: This soundtrack is my favorite music...
3,__label__2,Excellent Soundtrack: I truly like this soundt...
4,__label__2,"Remember, Pull Your Jaw Off The Floor After He..."


In [10]:
# Create sentiment column with 1 as positive (4/5 star), 0 negative (1,2 star)
reviews_df['sentiment'] = (reviews_df['rating'] == '__label__2')*1
reviews_df['gibberish'] = 0

There is a massive class imbalance with over 1 million real reviews and only 3767 gibberish reviews. We'll need to do some resampling.

In [11]:
gibb_df.rename(columns={'Response': 'review', 'Label': 'gibberish'}, inplace=True)

In [12]:
# Make new merged dataframe of just the reviews and whether they are gibberish
merged_df = pd.concat([gibb_df, reviews_df[['review', 'gibberish']]], ignore_index=True)
merged_df

,review,gibberish
0,ggg,1
1,hgghghghghghghhg,1
2,ufdhgjndfnvbhfdjvnjkmfgbdfg,1
3,dbdbdbd,1
4,dfgdfgd,1
...,...,...
1052338,Cheap and flimsy: This was bought for an event...,0
1052339,Total waste of money: This was a total waste o...,0
1052340,Whitmor budget garment rack: I purchased the W...,0
1052341,Serves its purpose: I bought this to put in my...,0


In [13]:
len(merged_df) - len(gibb_df) - len(reviews_df)

0

In [14]:
# FEATURE - Calculate entropy
def calculate_entropy(text):
    """Calculate Shannon entropy of the text to detect randomness."""
    if not text:
        return 0
    if not isinstance(text, str) or pd.isna(text):
        return 0  # Return 0 for NaN or non-string values
    text = str(text).lower()
    length = len(text)
    if length == 0:  # Handle empty strings
        return 0
    char_counts = Counter(text)
    entropy = -sum((count/length) * math.log2(count/length) for count in char_counts.values())
    return entropy

In [ ]:
# FEATURE - Can detect language
from langdetect import detect
import warnings

# Suppress langdetect warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

def detect_language(text):

    if not isinstance(text, str) or pd.isna(text) or len(text.strip()) < 3:
        return 'unknown'  # For NaN, empty, or very short text
    try:
        return detect(text)
    except:
        return 'unknown'  # Fallback for any detection errors

# Returns 0 if we can't find the langauge, 1 if we can
def cannot_detect_language(text):
    if text == 'unknown':
        return 1
    else:
        return 0

The next feature we'll look at is a flag of type of a few main alphabets. In our EDA we showed that outside of English, the most popular of our Coursera review languages mostly used a Latin or very close to Latin alphabet (somali, afrikaans, and tagalog for example use some variation of latin). We'll make flags for 
1. chinese
2. cyrillic (for russian, slovenian)
3. hangul (for korean)
4. Latin or latin variant

In [17]:
# FEATURE - Type of Alphabet (run time 3 minutes)
def detect_alphabets(text):

    if not isinstance(text, str) or not text:
        return {
            'Chinese': {'present': False, 'count': 0},
            'Cyrillic': {'present': False, 'count': 0},
            'Hangul': {'present': False, 'count': 0},
            'Latin': {'present': False, 'count': 0}
        }
    
    # Define Unicode ranges
    ranges = {
        'Chinese': (0x4E00, 0x9FFF),         # CJK Unified Ideographs
        'Cyrillic': (0x0400, 0x04FF),       # Basic Cyrillic
        'Hangul': (0xAC00, 0xD7AF),         # Hangul Syllables
        'Latin': [(0x0000, 0x007F),         # Basic Latin
                  (0x00A0, 0x00FF),         # Latin-1 Supplement
                  (0x0100, 0x017F)],        # Latin Extended-A
    }
    
    # Count characters per alphabet
    alphabet_counts = defaultdict(int)
    for char in text:
        char_code = ord(char)
        
        # Check Chinese
        if ranges['Chinese'][0] <= char_code <= ranges['Chinese'][1]:
            alphabet_counts['Chinese'] += 1
        
        # Check Cyrillic
        if ranges['Cyrillic'][0] <= char_code <= ranges['Cyrillic'][1]:
            alphabet_counts['Cyrillic'] += 1
        
        # Check Hangul
        if ranges['Hangul'][0] <= char_code <= ranges['Hangul'][1]:
            alphabet_counts['Hangul'] += 1
        
        # Check Latin (multiple ranges)
        for start, end in ranges['Latin']:
            if start <= char_code <= end:
                alphabet_counts['Latin'] += 1
                break  # Stop after first match
    
    # Build result dictionary
    result = {
        'Chinese': {'present': alphabet_counts['Chinese'] > 0, 'count': alphabet_counts['Chinese']},
        'Cyrillic': {'present': alphabet_counts['Cyrillic'] > 0, 'count': alphabet_counts['Cyrillic']},
        'Abakada': {'present': alphabet_counts['Abakada'] > 0, 'count': alphabet_counts['Abakada']},
        'Hangul': {'present': alphabet_counts['Hangul'] > 0, 'count': alphabet_counts['Hangul']},
        'Latin': {'present': alphabet_counts['Latin'] > 0, 'count': alphabet_counts['Latin']}
    }
    
    return result

def create_alphabet_tag_feature(df, review_col='review'):
    # Create alphabets column with dictionary structure above
    tqdm.pandas()
    df['alphabets'] = df[review_col].progress_apply(detect_alphabets)

    # Create one-hot tag of different alphabets
    df['has_chinese'] = df['alphabets'].apply(lambda x: x['Chinese']['present'])
    df['has_cyrillic'] = df['alphabets'].apply(lambda x: x['Cyrillic']['present'])
    df['has_abakada'] = df['alphabets'].apply(lambda x: x['Abakada']['present'])
    df['has_hangul'] = df['alphabets'].apply(lambda x: x['Hangul']['present'])
    df['has_latin'] = df['alphabets'].apply(lambda x: x['Latin']['present'])
    return df

In [ ]:
# 3x FEATURES - Total characters in review, word count of review, avg word length
def word_count(text):
    words = re.split(f'\s+', text.strip())
    word_count = len(words)
    return word_count

def char_count(text):
    return len(text)

def create_word_and_char_counts_feature(df, review_col='review'):
    tqdm.pandas()
    df['word_count'] = df[review_col].progress_apply(word_count)
    df['n_chars'] = df[review_col].progress_apply(char_count)
    return df

def get_avg_word_length(text):
    # avg word length
    words = re.split(f'\s+', text.strip())
    word_count = len(words)
    avg_word_length = sum(len(word) for word in words if word) / max(1, word_count) if words else 0
    return avg_word_length

def create_avg_word_length_feature(df, review_col='review'):
    tqdm.pandas()
    df['avg_word_length'] = df[review_col].progress_apply(get_avg_word_length)
    return df

In [ ]:
# FEATURE - Amount of Reptition
def get_max_repeated(text):
    max_repeats = max([sum(1 for _ in g) for _, g in itertools.groupby(text)] or [0])
    return max_repeates
def create_repetition_feature(df, review_col='review'):
    tqdm.pandas()
    df['max_repeated'] = df[review_col].progress_apply(get_max_repeated)
    return df

In [ ]:
# FEATURE - Punctuation Ratio
def get_punct_ratio(text):
    punct_count = sum(1 for c in text if c in '.,!?')
    punct_ratio = punct_count / max(1, char_length)
    return punct_ratio
def create_punct_ratio_feature(df, review_col='review'):
    df['punct_ratio'] = df[review_col].progress_apply(get_punct_ratio)
    return df

In [ ]:
# FEATURE - Contains common n-grams
# Step 1 - Build n-gram reference from sample of real reviews
def build_ngram_reference(texts, n=2, top_k=1000, sample_size=10000):
    """
    Build a set of common n-grams from a list of texts, assuming most are meaningful.
    WARNING - Must only be done on training set
    
    Parameters:
    - texts: List of text strings
    - n: N-gram size
    - top_k: Number of top n-grams to keep
    - sample_size: Number of texts to sample (to speed up)
    
    Returns:
    - Set of common n-grams
    """
    # Sample texts to avoid over-processing (e.g., 1.19M reviews)
    if len(texts) > sample_size:
        texts = np.random.choice(texts, sample_size, replace=False)
    
    # Generate n-grams
    ngrams = Counter()
    for text in tqdm(texts, desc="Building n-gram reference"):
        text = str(text).lower()
        for i in range(len(text) - n + 1):
            ngram = text[i:i+n]
            if not ngram.isspace():
                ngrams[ngram] += 1
    
    # Return top k most common n-grams
    return set([ngram for ngram, _ in ngrams.most_common(top_k)])

def create_ngram_coherence_feature(df, ngram_ref, review_col='review'):
    # N-gram coherence
    # Get reference set of ngrams
    text_lower = text.lower()
    total_ngrams = max(1, len(text_lower) - n + 1)
    valid_ngrams = sum(1 for i in range(total_ngrams) if text_lower[i:i+n] in ngram_ref)
    ngram_coherence = valid_ngrams / total_ngrams
    return ngram_coherence

In [ ]:
tagged_df = create_alphabet_tag_feature(merged_df.copy())
# create ngram_ref from training data only
ngram_ref = build_ngram_reference(df[review_col].dropna().tolist(), 
                                      n=2, top_k=1000, sample_size=10000)
tagged_df.head(2)

## FEATURE - 

In [ ]:
# 

def language_id

Existing Features
Entropy:
Calculate Shannon entropy of characters (as before).

Gibberish tends to have higher or flatter entropy; substantive reviews have structured patterns.

Code: Already in your toolkit (e.g., calculate_entropy).

Language Detection:
Use langdetect to get the language code.

Feature: Binary flag (1 if "unknown" or low-confidence language, 0 if recognized).

Code: detect_language from earlier.

Word Frequency:
Count known common words per language (e.g., "good," "bad" in English; "好" in Chinese).

Feature: Ratio of common words to total words.

Requires a dictionary per language (e.g., from NLTK or custom lists).

New Features
Word Count:
Number of words (split by spaces or language-specific tokenizers).

Gibberish often has fewer "words" or unrecognizable tokens.

Feature: Integer count or log(count).

Character Length:
Total characters in the review.

Very short reviews (e.g., "a") or overly long random strings may indicate gibberish.

Feature: Integer or normalized (length / max_length).

Repetition:
Measure character or word repetition (e.g., "aaa" or "good good good").

Feature: Max consecutive repeats or entropy of word frequency.

Code: Use collections.Counter on words/characters.

Punctuation Ratio:
Proportion of punctuation to alphanumeric characters.

Gibberish might over- or under-use punctuation (e.g., "!!!").

Feature: Float (punctuation_count / total_chars).

N-Gram Coherence:
Check if bigrams/trigrams match common patterns in each language.

Gibberish lacks typical n-gram structure (e.g., "th" in English vs. "xz").

Feature: Proportion of valid n-grams (requires language-specific n-gram lists).

Text Complexity:
Average word length or readability score (e.g., Flesch-Kincaid, simplified for multilingual).

Gibberish often has inconsistent or extreme word lengths.

Feature: Float (mean word length).

Embedding-Based Features (Advanced):
Use a pre-trained multilingual model (e.g., BERT, XLM-R) to get sentence embeddings.

Feature: Cosine similarity to a centroid of substantive reviews or anomaly score.

Requires more computation but leverages semantic coherence.



In [ ]:

# Feature extraction function
def extract_features(text):
    if not isinstance(text, str) or pd.isna(text):
        text = ""
    
    # Entropy
    length = len(text)
    if length == 0:
        entropy = 0
    else:
        char_counts = Counter(text.lower())
        entropy = -sum((count/length) * np.log2(count/length) for count in char_counts.values())
    
    # Language
    lang = detect(text) if length >= 3 else 'unknown'
    is_unknown = 1 if lang == 'unknown' else 0
    
    # Word count and common words (example for English)
    words = re.split(r'\s+', text.strip())
    word_count = len(words)
    common_words = {'good', 'great', 'bad', 'course', 'learn'}  # Expand per language
    common_word_ratio = sum(1 for w in words if w.lower() in common_words) / max(1, word_count)
    
    # Character length
    char_length = len(text)
    
    # Repetition
    max_repeats = max([sum(1 for _ in g) for _, g in itertools.groupby(text)] or [0])
    
    # Punctuation ratio
    punct_count = sum(1 for c in text if c in '.,!?')
    punct_ratio = punct_count / max(1, char_length)
    
    return [entropy, is_unknown, word_count, common_word_ratio, char_length, max_repeats, punct_ratio]


In [ ]:
import pandas as pd
import numpy as np
from langdetect import detect
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import re

# Feature extraction function
def extract_features(text):
    if not isinstance(text, str) or pd.isna(text):
        text = ""
    
    # Entropy
    length = len(text)
    if length == 0:
        entropy = 0
    else:
        char_counts = Counter(text.lower())
        entropy = -sum((count/length) * np.log2(count/length) for count in char_counts.values())
    
    # Language
    lang = detect(text) if length >= 3 else 'unknown'
    is_unknown = 1 if lang == 'unknown' else 0
    
    # Word count and common words (example for English)
    words = re.split(r'\s+', text.strip())
    word_count = len(words)
    common_words = {'good', 'great', 'bad', 'course', 'learn'}  # Expand per language
    common_word_ratio = sum(1 for w in words if w.lower() in common_words) / max(1, word_count)
    
    # Character length
    char_length = len(text)
    
    # Repetition
    max_repeats = max([sum(1 for _ in g) for _, g in itertools.groupby(text)] or [0])
    
    # Punctuation ratio
    punct_count = sum(1 for c in text if c in '.,!?')
    punct_ratio = punct_count / max(1, char_length)
    
    return [entropy, is_unknown, word_count, common_word_ratio, char_length, max_repeats, punct_ratio]

# Load labeled product review dataset
product_df = pd.read_csv('product_reviews_labeled.csv')  # Assume columns: 'review', '

# Extract features
X = np.array([extract_features(review) for review in product_df['review']])
y = product_df['is_gibberish'].values  # 1 = gibberish, 0 = not gibberish

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Apply to course reviews (merged_df)
course_features = np.array([extract_features(review) for review in merged_df['reviews']])
course_predictions = clf.predict(course_features)
merged_df['is_gibberish'] = course_predictions

# Save results
merged_df.to_parquet('course_reviews_classified.parquet')
print("Course reviews classified and saved.")